#Deuxième partie : statistiques descriptives et analyses préparatoires

In [2]:
# on réimporte les librairies
from pyspark.sql.functions import * #ou import pyspark.sql.functions as psf
from pyspark.sql.types import *

In [3]:
# on réimporte le dataframe sauvegardé enfin de partie 1 au format "parquet"
df_clean = spark.read.load("/FileStore/parquet/airbnb_part_1")
display(df_clean.limit(50))

id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,jurisdiction_names,cancellation_policy,calculated_host_listings_count,reviews_per_month,geolocation,features
1002049,https://www.airbnb.com/rooms/1002049,20170407214050,2017-04-08T00:00:00.000+0000,Central room with private balcony!,Nice and bright room with balcony,"ABOUT ROOM This is a comfortable, modern and brand-new room measuring 23 sq m. in the main city center next to Las Ramba Raval!. Bed it's size is 200 x 140 so it's perfect fits for couples! Also room have cozy balcony, where you can relax and have a breakfast. In the shared kitchen only for breakfast, you will find all the stuff you may need for cooking. Fresh blankets are also included. Rental Towels /3€*1 THE AREA Room five min from main street Las Ramblas. In the neighborhood you will find interesting museums, shops, art galleries and cozy bars. Moreover is just few minutes from famous La Boqueria market, where you can get fresh fruits, fish, and even more... DISTANCE: Las Ramblas - 5 min walking. Plaza Catalunya - 7 min walking. La Boqueria - 5 min walking. Palau Guell - 7 min walking. Plaza Real - 7 min walking. Beach - 15 min walking HOW TO FIND APARTMENT El prat airport to take AeroBus to Plaza Catalunya, and then is just 10 min walking to the apartment. Please don't h","Nice and bright room with balcony ABOUT ROOM This is a comfortable, modern and brand-new room measuring 23 sq m. in the main city center next to Las Ramba Raval!. Bed it's size is 200 x 140 so it's perfect fits for couples! Also room have cozy balcony, where you can relax and have a breakfast. In the shared kitchen only for breakfast, you will find all the stuff you may need for cooking. Fresh blankets are also included. Rental Towels /3€*1 THE AREA Room five min from main street Las Ramblas. In the neighborhood you will find interesting museums, shops, art galleries and cozy bars. Moreover is just few minutes from famous La Boqueria market, where you can get fresh fruits, fish, and even more... DISTANCE: Las Ramblas - 5 min walking. Plaza Catalunya - 7 min walking. La Boqueria - 5 min walking. Palau Guell - 7 min walking. Plaza Real - 7 min walking. Beach - 15 min walking HOW TO FIND APARTMENT El prat airport to take AeroBus to Plaza Catalunya, and then is just 10 min walkin",none,"Raval neighborhood is colorful,with a real mix of amazing cultures. where you can enjoy music, food, and some public rituals of defferent religions. Don't be surprice for the exitic people around they are our neighbors from india, pakistan,morocos, philipins etc :)","The flat has no elevator and it is a 3th floor. Is very important to understand that we rent one of the rooms with shared bathroom an kitchen * just for breakfast / Is NOT a full apartment . your reservation is included 2 towels/ one each person. Extra towels (3€ * 1) Please bring your hairdryer. We have a DOG labrador

### Les doublons

In [5]:
# on supprime les doublons : pas besoin de supprimer sur toutes les colonnes mais uniquement sur des colonnes "identifiantes" d'une annonce :
# on peut prendre id, name, host name, street (un propriétaire peut pas avoir 2 annonces ayant même id, même nom exactement dans la même rue )
df_clean = df_clean.dropDuplicates(subset=["id", "name", "host_name", "street"])

In [6]:
#print(df_clean.count())
#output : 494951

### Quelques colonnes inutiles d'office

In [8]:
# on peut déjà supprimer certaines colonnes qui nous serviront pas :
# - les URL
for colname in df_clean.columns :
  if "url" in colname.lower():
    df_clean = df_clean.drop(colname)

# - les colonnes de date en passant par le type Timestamp
for f in df_clean.schema.fields:
  if f.dataType == TimestampType():
    df_clean = df_clean.drop(f.name)
    
# - les colonnes concernant les infos de scraping
for colname in df_clean.columns :
  if "scrap" in colname.lower():
    df_clean = df_clean.drop(colname)

### Des lignes dont on ne fera rien...
Avant d'aller plus loin, on va supprimer les lignes n'ayant pas de `review score rating`, de `price` ou de `reviews per month`. En effet, qu'on décide de s'intéresser au prix (pour par exemple construire un outil de conseil de prix optimal pour une annonce donnée) ou à la note globale (pour trouver les facteurs déterminants d'une location qui plaît aux users) ou encore au nombre d'avis par mois (qui est proxy pour le nombre de visiteurs mensuel), la note globale reste une donnée fondamentale puisqu'une annonce n'ayant pas de review n'a pas été testée et donc on ne peut pas "valider" le prix qui a été annoncé par le loueur. Par ailleurs, les annonces sans reviews peuvent aussi être fake.

In [10]:
# suppression lignes sans Review Scores Rating, ça concerne un peu plus de 120000 observations
df_clean = df_clean.dropna(subset=["review_scores_rating", "price", "reviews_per_month"], how="any")
#df_clean.count()
#output : 362417

```Python
# on regarde les valeurs nan et null

# pour les nan y a un petit problème avec les types timestamp qui ne peuvent avoir des NaN (mais des NaT)
# on filtre donc les colonnes sur les types
from pyspark.sql.types import TimestampType
nans = df_clean.select([count(when(isnan(f.name), f.name)).alias(f.name) for f in df_clean.schema.fields if f.dataType != TimestampType()]).toPandas()
nans
```
Il n'y a pas de NaN dans le dataset, uniquement des None/null.

In [12]:
# pour les null on utilise column.isNull()
#nulls = df_clean.select([count(when(col(colname).isNull(), colname)).alias(colname) for colname in df_clean.columns]).toPandas()
#nulls

Il reste pas mal de valeurs manquantes mais on va certainement modifier les variables et n'en conserver qu'une partie donc on gérera les valeurs manquantes restantes après ces opérations de pré-traitement et avant la modélisation. Dans un premier temps, on va regarder les valeurs manquantes pour les variables ayant relativement peu de valeurs manquantes et/ou qui nous intéressent particulièrement :
- Description
- Accommodates
- Extra People
- Property Type
- Bathrooms
- Bedrooms
- Beds
- Host Response Time
- Host Response Rate
- City/Market

In [14]:
# Description : 65 valeurs manquantes
#display(df_clean.filter(col("description").isNull()))

# Pas de traitement possible, certaines lignes on des informations dans d'autres variables descriptives comme Host About ou encore Space ou Summary
# toutefois, peu de d'information à récupérer dans l'ensemble donc on laisse tomber cette idée

In [15]:
# Accommodates : 55 valeurs manquantes
#display(df_clean.filter(col("accommodates").isNull()))

# On se rend compte que ceux pour lesquels accommodates est manquant ont toujours la variable Beds :
# on peut pour faire simple en estimant que les lits sont doubles et multiplier par 2 pour avoir un nb de places : très légère surestimation possible mais pour ces 55 observations, on s'en remettra
# notez l'utilisation de coalesce pour prendre les valeurs d'une autre colonne lorsque celles de la première colonne valent null
df_clean = df_clean.withColumn("accommodates", coalesce(col("accommodates"), 2*col("beds")))

In [16]:
# Extra People : 10 valeurs manquantes
#display(df_clean.filter(col("extra_people").isNull()))

# On peut affecter extra_people à 0 car on a à chaque fois (presque...) accommodates=guests included
df_clean = df_clean.fillna(0, subset=["extra_people"])

In [17]:
# Property Type : 4 valeurs manquantes
#display(df_clean.filter(col("property_type").isNull()))

# On peut récupérer le Property Type en lisant la description et affecter une valeur correspondante (il faut checker les modalités de property type meme si on reviendra dessus plus loin)
# 10758 cheap hotel with shared bathroom => modalité Hostel
# 7731797 Classy Studio => modalité Apartment
# 11345294 Boutique hotel => modalité Boutique hotel
# 296629 Condominium => modalité Condominium
df_clean = df_clean.withColumn("property_type",
                    when(col("id") == '10758', 'Hostel') \
                    .when(col("id") == '7731797', 'Apartment') \
                    .when(col("id") == '11345294', 'Boutique hotel') \
                    .when(col("id") == '296629', 'Condominium') \
                    .otherwise(col("property_type")))

In [18]:
# Bathrooms : 1006 valeurs manquantes
#display(df_clean.filter(col("bathrooms").isNull()))

# pour commencer si c'est un appartement entier on peut fixer à 1 sdb
# mais comme la médiane est de 1 on va faire simple et imputer 1 pour tout le monde
#df_clean.approxQuantile("bathrooms", [0.5], 0.05)
df_clean = df_clean.fillna(1, subset=["bathrooms"])

In [19]:
# Bedrooms : 443 valeurs manquantes
#display(df_clean.filter(col("bedrooms").isNull()))

#s'il s'agit d'une chambre on peut mettre bedrooms = 1
df_clean = df_clean.withColumn("bedrooms", when((col("bedrooms").isNull())&(col("room_type")=="Private room"), lit(1)).otherwise(col("bedrooms")))

#si c'est un studio ou un loft, on peut mettre bedrooms = 0
df_clean = df_clean.withColumn("bedrooms",
                               when((col("bedrooms").isNull())&(
                                 lower(concat(
                                   coalesce(col("name"),lit(' ')),
                                   coalesce(col("summary"),lit(' ')),
                                   coalesce(col("space"),lit(' ')),
                                   coalesce(col("description"),lit(' '))
                                 )).rlike("studio|loft")),lit(0)) \
                               .otherwise(col("bedrooms")))

# on peut faire une estimation grossière si on a le nombre de lits : floor(beds/2+1)
# 1 lit = 1 chambre, 2 ou 3 lits = 2 chambres, 4 ou 5 lits = 3 chambres, 6 ou 7 lits = 4 chambres, etc...
df_clean = df_clean.withColumn("bedrooms", coalesce(col("bedrooms"), floor(col("beds")/2 + 1)))

# pour les dernières lignes manquantes, on peut passer par accommodates en estimant grossièrement qu'il y a :
# environ 1 chambre pour 2 places avec floor(accommodates/2)
df_clean = df_clean.withColumn("bedrooms", coalesce(col("bedrooms"), floor(col("accommodates")/2)))

In [20]:
# Beds : 406 valeurs manquantes
#display(df_clean.filter(col("beds").isNull()))

# comme pour les chambres on peut faire une estimation grossière en considérant qu'il y a un maximum de lits doubles
# et on complète pour les impairs avec un lit simple en prenant la partie entière supérieur (ceil)
df_clean = df_clean.withColumn("beds", coalesce(col("beds"), ceil(col("accommodates")/2)))

In [21]:
# Host Response Time : plus de 50000 valeurs manquantes
#display(df_mod.where(col("host_response_time").isNull()))

# On peut pas vraiment utiliser d'autres variables pour imputer
# On ne peut pas non plus imputer la modalité la plus fréquentes par exemple car on modifierait trop la structure des données
# On va donc créer une modalité : not available quitte à perdre le caractère ordonné de cette variable catégorique

df_clean = df_clean.fillna("not available", subset=["host_response_time"])

In [22]:
# Host Response Rate : plus de 50000 valeurs manquantes
#display(df_clean.where(col("host_response_rate").isNull()))

# on  a pas d"infos supplémentaire utilisable donc on va imputer la médiane pour simplifier
#conversion en integer
df_clean = df_clean.withColumn("host_response_rate", col("host_response_rate").cast(IntegerType()))
# calcul de la médiane : on obtient 100
#df_clean.approxQuantile("host_response_rate", [0.5], 0.05)

# imputation de la médiane
df_clean = df_clean.fillna(100, subset=["host_response_rate"])

In [23]:
# Market : 3048 valeurs manquantes
#display(df_clean.where(col("market").isNull()))

# On s'intéresse à market pour avoir une maille géographique plus fine que country mais sans avoir non plus trop de modalités (city en a 4000)
# Il faut par contre gérer les 3000 nans

# Pour commencer on va affecter city lorsque market est nul
df_clean = df_clean.withColumn("market", coalesce(col("market"), col("city")))

# et il se trouve que ça suffit ! :)
#df_clean.where(col("market").isNull()).count()
# output : 0 manquants restants

### Gestion des variables de description de l'annonce (texte)

Tout d'abord, on ne va pas faire d'analyse textuelle, on reviendra peut-être là dessus en fonction du temps. Par contre on peut tirer des informations de ces descriptions qui pourraient être utiles pour la modélisation : longueur (nombre de mots ou caractères), nombre de descriptions remplies par exemple.

In [25]:
# Première étape : identifier et récupérer les variables concernées 
from pyspark.sql.types import StringType
[f.name for f in df_clean.schema.fields if f.dataType == StringType()]

Out[18]: ['id',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'host_name',
 'host_location',
 'host_about',
 'host_response_time',
 'host_acceptance_rate',
 'host_neighbourhood',
 'host_verifications',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'property_type',
 'room_type',
 'bed_type',
 'amenities',
 'square_feet',
 'price',
 'weekly_price',
 'monthly_price',
 'calendar_updated',
 'has_availability',
 'first_review',
 'last_review',
 'license',
 'jurisdiction_names',
 'cancellation_policy',
 'calculated_host_listings_count',
 'reviews_per_month',
 'geolocation',
 'features']

In [26]:
# les variables concernées sont notamment les premières : de Summary à House Rules sauf Experiences Offered
# on peut y ajouter Host About

# La colonne description reprend généralement plus ou moins les 2 colonnes summary et space donc on va compter que la description
# pour les autres variables, on va se contenter de faire un compteur des descriptions effectuées par l'hôte
# finalement on aura : le nb de mot dans la description et le nb de types de descriptions renseignées

desc_var = ['summary', 'space', 'description', 'neighborhood_overview',
            'notes', 'transit', 'access', 'interaction', 'house_rules',
            'host_about']

# on compte la longueur de la variable Description
df_mod = df_clean.withColumn('description_len', size(split(col('description'), ' ')))

# pour toutes les variables, on les remplace par 1 si elles sont non-vides
for var in desc_var:
  df_mod = df_mod.withColumn(var, when(col(var).isNull(),0).otherwise(1))
  
#puis on les somme pour avoir le nombre de non_vides
from operator import add
from functools import reduce
df_mod = df_mod.withColumn('nb_desc',reduce(add, [col(var) for var in desc_var]))

# puis on supprime les colonnes
for var in desc_var:
  df_mod = df_mod.drop(var)

#display(df_mod)

### Dummies à la main

On regarde ensuite un certain nombre de variables qui sont intéressantes et qu'il faudra transformer en indicatrices.

#### Variable `Features`

In [29]:
# La variable features
#display(df_mod.groupBy("features").count())

In [30]:
# L'immense majorité des hôtes ont des photos de profil. On ne crée des indicatrices que pour des modalités plus discriminantes :
# l'exactitude de la localisation, si l'identité de l'hôte est vérifiée, si la réservation est instantanée et si l'hôte est un super-hôte.
df_mod = df_mod.withColumn("is_location_exact", when(df_clean.features.contains("Is Location Exact"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("host_identity_verified", when(df_clean.features.contains("Host Identity Verified"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("instant_bookable", when(df_clean.features.contains("Instant Bookable"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("host_is_superhost", when(df_clean.features.contains("Host is Superhost"), lit(1)).otherwise(lit(0)))

#### Variable `Cancellation Policy`

In [32]:
# Cancellation Policy
#display(df_mod.groupBy("cancellation_policy").count())

In [33]:
# Différents critères et qui ont évolué, on va essayer de reconstituer trois critères : strict, modéré, flexible
# Deux variables indicatrices nécéssaire pour reconstituer cette information

# pour strict, on prend les modalités qui contiennent strict et la modalité no_refunds : 
# notez l'utilisation de rlike pour pouvoir faire la condition sur une regex
df_mod = df_mod.withColumn("cancel_strict", when(df_mod["cancellation_policy"].rlike("strict|no_refunds"), lit(1)).otherwise(lit(0)))

# pour moderate, on prend les modalités qui contiennent moderate
df_mod = df_mod.withColumn("cancel_moderate", when(df_mod["cancellation_policy"].contains("moderate"), lit(1)).otherwise(lit(0)))

#### Variable `Property Type`

In [35]:
#display(df_mod.groupBy("property_type").count())

In [36]:
# Nombreux types de résidence. On va essayer de créer quatre catégories :
# - Maison ou équivalent
# - Appartement ou équivalent
# - Chambre d'hôte/hôtel
# - Autres
# On aura donc besoin de trois variables indicatrices pour les décrire : apartment, house, hostel
df_mod = df_mod.withColumn("apartment", when(df_mod["property_type"].rlike("Condominium|Apartment|Loft"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("house", when(df_mod["property_type"].rlike("Townhouse|Villa|House|Chalet|Casa"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("hostel", when(df_mod["property_type"].rlike("Bed & Breakfast|Guesthouse|Hostel"), lit(1)).otherwise(lit(0)))

#### Variable `Amenities`

On peut utiliser l'API d'OpenDataSoft où les modalités sont affichées et classées par fréquences pour yvoir plus clair.

Quelques remarques :
 - pour commencer, il y a beaucoup de modalités il va donc falloir faire un petit ménage
 - WiFi (à regrouper avec Internet), Kitchen, Heating et Essentials ont plus de 420000 observations dans le dataset initial c'est-à-dire que quasi tout le monde a ses équipements
 - il y a un problème de casse avec des modalités identiques à la majuscule près
 - un certain nombre de modalités ont des fréquences très faibles moins de 1000 occurences
 - des modalités peuvent être regroupées sous un même thème

In [39]:
# on se débarrasse de la casse
df_mod = df_mod.withColumn("amenities", lower(col("amenities")))

# il y a aussi 2 modalités problématiques : translation missing: en.hosting_amenity_{49,50}
#df_mod = df_mod.withColumn('Amenities', regexp_replace('Amenities', ',translation missing: en.hosting_amenity_49', ''))
#df_mod = df_mod.withColumn('Amenities', regexp_replace('Amenities', ',translation missing: en.hosting_amenity_50', ''))

#display(df_mod.groupBy("amenities").count())

In [40]:
# Finalement, on créera les indicatrices suivantes :
df_mod = df_mod.withColumn("tv", when(df_mod["amenities"].rlike("tv"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("cable_tv", when(df_mod["amenities"].rlike("cable"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("free_parking", when(df_mod["amenities"].rlike("free parking"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("pool", when(df_mod["amenities"].rlike("pool"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("gym", when(df_mod["amenities"].rlike("gym"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("air_conditioning", when(df_mod["amenities"].rlike("air conditioning"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("elevator", when(df_mod["amenities"].rlike("elevator in building"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("detectors", when(df_mod["amenities"].rlike("detector|fire extinguisher"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("appliances", when(df_mod["amenities"].rlike("dryer|washer"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("pets", when(df_mod["amenities"].rlike("pet|dog|cat"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("disable", when(df_mod["amenities"].rlike("disable|wheelchair|wide|accessible"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("kids", when(df_mod["amenities"].rlike("kid|baby"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("smoking", when(df_mod["amenities"].rlike("smoking"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("smart_clothes", when(df_mod["amenities"].rlike("iron|hanger"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("fireplace", when(df_mod["amenities"].rlike("fireplace"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("easy_check_in", when(df_mod["amenities"].rlike("check-in"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("breakfast", when(df_mod["amenities"].rlike("breakfast"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("tub", when(df_mod["amenities"].rlike("tub"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("events", when(df_mod["amenities"].rlike("event"), lit(1)).otherwise(lit(0)))
df_mod = df_mod.withColumn("privacy", when(df_mod["amenities"].rlike("private"), lit(1)).otherwise(lit(0)))

In [41]:
# On peut aussi créer une variable qui compte le nombre d'équipements global
df_mod = df_mod.withColumn("amenities_nb", size(split(col("amenities"), ",")))

#### Variable `Security Deposit`

In [43]:
df_mod = df_mod.withColumn("is_depot", when(df_mod["security_deposit"] > 0, lit(1)).otherwise(lit(0)))

#### Variable `Experiences Offered`

In [45]:
df_mod = df_mod.withColumn("experiences", when(df_mod["experiences_offered"]=="none", lit(0)).otherwise(lit(1)))

### Quelques analyses descriptives  
La base est large, on va regarder si on observe de grandes différences entre pays ou non.

In [47]:
# On convertit le price (qui est un string) en integer
df_mod = df_mod.withColumn("price", col("price").cast(IntegerType()))

# on met des 0 quand cleanin_fee vaut null
df_mod = df_mod.fillna(0, subset=["cleaning_fee"])

# puis on calcul le prix total incluant les frais de ménage
df_mod = df_mod.withColumn("total_price", (col("price")+col("cleaning_fee")).cast(IntegerType()))

df_mod.dtypes

Out[31]: [('id', 'string'),
 ('name', 'string'),
 ('experiences_offered', 'string'),
 ('host_id', 'int'),
 ('host_name', 'string'),
 ('host_location', 'string'),
 ('host_response_time', 'string'),
 ('host_response_rate', 'int'),
 ('host_acceptance_rate', 'string'),
 ('host_neighbourhood', 'string'),
 ('host_listings_count', 'int'),
 ('host_total_listings_count', 'int'),
 ('host_verifications', 'string'),
 ('street', 'string'),
 ('neighbourhood', 'string'),
 ('neighbourhood_cleansed', 'string'),
 ('neighbourhood_group_cleansed', 'string'),
 ('city', 'string'),
 ('state', 'string'),
 ('zipcode', 'string'),
 ('market', 'string'),
 ('smart_location', 'string'),
 ('country_code', 'string'),
 ('country', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('property_type', 'string'),
 ('room_type', 'string'),
 ('accommodates', 'double'),
 ('bathrooms', 'double'),
 ('bedrooms', 'bigint'),
 ('beds', 'double'),
 ('bed_type', 'string'),
 ('amenities', 'string'),
 ('square_feet', 'string'),
 ('price', 'int'),
 ('weekly_price', 'string'),
 ('monthly_price', 'string'),
 ('security_deposit', 'double'),
 ('cleaning_fee', 'double'),
 ('guests_included', 'int'),
 ('extra_people', 'double'),
 ('minimum_nights', 'int'),
 ('maximum_nights', 'int'),
 ('calendar_updated', 'string'),
 ('has_availability', 'string'),
 ('availability_30', 'int'),
 ('availability_60', 'double'),
 ('availability_90', 'double'),
 ('availability_365', 'double'),
 ('number_of_reviews', 'int'),
 ('first_review', 'string'),
 ('last_review', 'string'),
 ('review_scores_rating', 'double'),
 ('review_scores_accuracy', 'double'),
 ('review_scores_cleanliness', 'int'),
 ('review_scores_checkin', 'int'),
 ('review_scores_communication', 'double'),
 ('review_scores_location', 'double'),
 ('review_scores_value', 'double'),
 ('license', 'string'),
 ('jurisdiction_names', 'string'),
 ('cancellation_policy', 'string'),
 ('calculated_host_listings_count', 'string'),
 ('reviews_per_month', 'string'),
 ('geolocation', 'string'),
 ('features', 'string'),
 ('description_len', 'int'),
 ('nb_desc', 'int'),
 ('is_location_exact', 'int'),
 ('host_identity_verified', 'int'),
 ('instant_bookable', 'int'),
 ('host_is_superhost', 'int'),
 ('cancel_strict', 'int'),
 ('cancel_moderate', 'int'),
 ('apartment', 'int'),
 ('house', 'int'),
 ('hostel', 'int'),
 ('tv', 'int'),
 ('cable_tv', 'int'),
 ('free_parking', 'int'),
 ('pool', 'int'),
 ('gym', 'int'),
 ('air_conditioning', 'int'),
 ('elevator', 'int'),
 ('detectors', 'int'),
 ('appliances', 'int'),
 ('pets', 'int'),
 ('disable', 'int'),
 ('kids', 'int'),
 ('smoking', 'int'),
 ('smart_clothes', 'int'),
 ('fireplace', 'int'),
 ('easy_check_in', 'int'),
 ('breakfast', 'int'),
 ('tub', 'int'),
 ('events', 'int'),
 ('privacy', 'int'),
 ('amenities_nb', 'int'),
 ('is_depot', 'int'),
 ('experiences', 'int'),
 ('total_price', 'int')]

In [48]:
# On va déterminer un prix par personne en utilisant les variables accomodates, guests_included et extra_price
@udf('integer')
def price_per_person(price, accom, guest, extra):
  if accom <= guest: # si le nb de guests est le même que le nombre de place alors pas de pbm
    return int(price/accom)
  else:   #sinon, on ajoute au prix le montant extra * nb de guests possibles non inclus (càd places - guests inclus)
      tot_price = price + (accom-guest)*extra
      return int(tot_price/accom)
  
df_mod = df_mod.withColumn("price_per_person", price_per_person("total_price", "accommodates", "guests_included", "extra_people"))

In [49]:
#display(df_mod.groupBy("country").agg(avg("price_per_person")))

In [50]:
#display(df_mod.groupBy("country").agg(avg("review_scores_rating")))

On a des différences importantes en fonction des pays, il semble nécessaire de contrôler par cette variable

In [52]:
#display(df_mod.groupBy("experiences_offered").agg({"review_scores_rating" : 'avg', "reviews_per_month" : 'avg', "price_per_person" : 'avg'}))

### Création de dummies avec OneHotEncoder

On gère les pays (variable `country`), les villes (la variable `market` plus exactement) et la variable `host_response_time` avec OneHotEncoder.

La variable `market` nécessite toutefois un petit nettoyage des modalités car il y en plus de 300 et surtout il y a des doublons.

#### Nettoyage de `Country`

In [55]:
# Modalités
#display(df_mod.groupBy("country").count())

In [56]:
# On crée une modalité other avec les pays qui ont une fréquence faible (on fixe un seuil à 2000 observations pour inclure la suisse)
tmp = df_mod.groupBy("country").count().where(col("count") < 2000).select("country") #on récupère les markets avec freq < 50
tmp_list = tmp.collect() # on passe en liste avec collect
tmp_str = ('|').join([row.country for row in tmp_list]) # on récupère les valeurs des pays et on transforme en string avec séparateur |

# Plus qu'à modifier en other avec un rlike
df_mod = df_mod.withColumn("country", when(col("country").rlike(tmp_str), "Other").otherwise(col("country")))

#df_mod.select("country").distinct().count()
# output : 16 modalités restantes

#### Nettoyage de `Market`

In [58]:
# Modalités (trier par ordre alhabétique pour bien voir les doublons)
#display(df_mod.groupBy("market").count())

In [59]:
# On regroupe les modalités qui doivent l'être
df_mod = df_mod.withColumn("market",
                           when(col("market").contains("Antwerp"), "Antwerp") \
                           .when(col("market").contains("Bondi"), "Bondi") \
                           .when(col("market").contains("Brooklyn"), "New York") \
                           .when(col("market").contains("Brussel"), "Brussels") \
                           .when(col("market").contains("Chêne"), "Chêne") \
                           .when(col("market").contains("Los Angeles"), "Los Angeles") \
                           .when(col("market").contains("Fitzroy"), "Fitzroy") \
                           .when(col("market").contains("Genève"), "Geneva") \
                           .when(col("market").contains("benhavn"), "Copenhagen") \
                           .when(col("market").contains("Lazio"), "Lazio") \
                           .when(col("market").contains("Paris"), "Paris") \
                           .when(col("market").contains("Melbourne"), "Melbourne") \
                           .when(col("market").contains("Portland"), "Portland") \
                           .when(col("market").contains("Roma"), "Rome") \
                           .when(col("market").contains("Québec"), "Quebec") \
                           .when(col("market").contains("Washington"), "D.C.") \
                           .when(col("market").contains("Other"), "Other") \
                           .otherwise(col("market"))
                          )

#df_mod.select("market").distinct().count()
# output : 283 modalités restantes

In [60]:
# Reste à mettre dans la modalité other toutes celles qui ont une fréquence très faible (on fixe un seuil à 50 observations)
tmp = df_mod.groupBy("market").count().where(col("count") < 50).select("market") #on récupère les markets avec freq < 50
tmp_list = tmp.collect() # on passe en liste avec collect
tmp_str = ('|').join([row.market for row in tmp_list]) # on récupère les valeurs des pays et on transforme en string avec séparateur |

# Plus qu'à modifier en other avec un rlike
df_mod = df_mod.withColumn("market", when(col("market").rlike(tmp_str), "Other").otherwise(col("market")))

#df_mod.select("market").distinct().count()
# output : 52 modalités restantes

In [61]:
# Résultats de ce petit ménage
#display(df_mod.groupBy("market").count())

#### StringIndexer et OneHotEncoder sur les 3 variables concernées

In [63]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

liste = ["room_type", "bed_type", "market", "country", "host_response_time"]

for var in liste:
  # On passe les caractères en numériques avec StringIndexer
  indexer = StringIndexer(inputCol=var, outputCol=var+"_index").fit(df_mod)
  df_mod = indexer.transform(df_mod)

  
# On passe les vecteurs numériques en vecteurs de booléens
encoder = OneHotEncoderEstimator(inputCols=[var + "_index" for var in liste],
                                 outputCols=[var + "_dummies" for var in liste])
model = encoder.fit(df_mod)
df_mod = model.transform(df_mod)

In [64]:
# restriction aux variables qui nous intéressent
liste_var = ["country", "description_len", "nb_desc", "apartment", "house", "hostel", "cancel_strict", "cancel_moderate", "is_location_exact", "host_identity_verified", "instant_bookable", "host_is_superhost", "host_response_rate", "bathrooms", "accommodates", "bedrooms", "price_per_person", "extra_people", "minimum_nights", "availability_365", "number_of_reviews", "review_scores_rating", "reviews_per_month", "amenities_nb", "tv", "cable_tv", "free_parking", "pool", "gym", "air_conditioning", "elevator", "detectors", "appliances", "pets", "disable", "kids", "smoking", "smart_clothes", "fireplace", "easy_check_in", "breakfast", "tub", "events", "privacy", "is_depot", "experiences", "room_type_dummies", "bed_type_dummies", "market_dummies", "country_dummies", "host_response_time_dummies"]

df_end = df_mod.select(liste_var)

On enregistre la table nettoyée au format parquet :

In [66]:
df_end.write.format("parquet").mode("overwrite").save('/FileStore/parquet/airbnb_part_2')